In [1]:
import pandas as pd

In [2]:
def read_enedis(file, name_ts):
    df = pd.read_excel(file, skiprows=15)
    df.drop(labels=['Unnamed: 1'], axis=1, inplace=True)
    df.dropna(inplace=True)
    df = df.melt(id_vars='Date', var_name='n_half_hour', value_name=name_ts).sort_values(['Date', 'n_half_hour'])
    df['hour'] = (df.n_half_hour - 1) // 2
    df['minutes'] = ((df.n_half_hour - 1) % 2) * 30
    df.index = pd.to_datetime(df.Date.astype(str)+' '+df.hour.astype(str).apply(lambda x: '0'*(2-len(x))+x) +':'+df.minutes.astype(str).apply(lambda x: '0'*(2-len(x))+x), 
                              dayfirst=True,
                              format='mixed')
    df = df[[name_ts]]
    return df

In [7]:
for year in range(2015, 2021):
    TR_df = read_enedis(f'Temp_Reelles_{year}.xls', 'TR')
    TN_df = read_enedis(f'TN_{year}.xls', 'TN')
    T_df = pd.concat([TN_df, TR_df], axis=1)
    T_df['TRS'] = T_df.TR.where(T_df.TR < 15, T_df.TN)
    T_df['deltaT'] = T_df.TR - T_df.TN
    T_df['deltaTS'] = T_df.TRS - T_df.TN
    T_df.index.name = 'Timestamp'
    T_df.resample('h').mean().round(1).to_excel(f'{year}.xlsx')